In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode_outer,explode, posexplode, posexplode_outer
spark = SparkSession.builder \
    .appName("ProductCategoryPairs") \
    .getOrCreate()
'''
Датафрейм в качестве примера.
Ранее с pySpark работать не доводилось, так что не обязуюсь сказать, что решение идеально.
'''
data = [(["Product1"], ["Category1", "Category2"]),
        (["Product2"], ["Category1"]),
        (["Product3"], None),
        (["Product4", "Product5"], ["Category3"]),
        (None, ["Category6"]),
       (["Product7", "Product8"], None),]
schema = ["products", "categories"]
dataframe = spark.createDataFrame(data, schema)
dataframe.show()


+--------------------+--------------------+
|            products|          categories|
+--------------------+--------------------+
|          [Product1]|[Category1, Categ...|
|          [Product2]|         [Category1]|
|          [Product3]|                NULL|
|[Product4, Product5]|         [Category3]|
|                NULL|         [Category6]|
|[Product7, Product8]|                NULL|
+--------------------+--------------------+



In [3]:
def get_pairs_df(df):
    df_exploded1 = df.select('categories', explode("products").alias("product"))
    df_exploded2 = df_exploded1.select('product', explode_outer("categories").alias("category"))
    return df_exploded2

In [4]:
pairs_df=get_pairs_df(dataframe)
pairs_df.show()

+--------+---------+
| product| category|
+--------+---------+
|Product1|Category1|
|Product1|Category2|
|Product2|Category1|
|Product3|     NULL|
|Product4|Category3|
|Product5|Category3|
|Product7|     NULL|
|Product8|     NULL|
+--------+---------+

